### **In this notebook. we're going to cover some of the most fundamental concepts of tensors using TensorFlow**

**Most specifically, we're going to cover:**
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (way to speed up regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Excercises

## **Introduction to Tensors**

### **Creating tensors with tf.constant()**

In [66]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.10.0


In [67]:
# Create tensors with tf.contant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [68]:
# Check the number of dimensions of a tensor
scalar.ndim

0

In [69]:
# Creace a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10])>

In [70]:
# Check the number of dimensions of a vector
vector.ndim

1

In [71]:
# Create a matrix
matrix = tf.constant([[10,7],
                      [7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]])>

In [72]:
# Check the number of dimensions of a matrix
matrix.ndim

2

In [73]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # specify the data type
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [74]:
# Check the number of dimensions of an another_matrix
another_matrix.ndim

2

In [75]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                      [[7, 8, 9],
                       [10, 11, 12]],
                      [[13, 14, 15],
                       [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]])>

In [76]:
# Check the number of dimensions of a tensor
tensor.ndim

3

**What we've created so far:**

* **Scalar:** a single number
* **Vector:** a number with direction
* **Matrix:** a 2-dimentional array of numbers
* **Tensor:** an n-dimentional array of numbers

### **Creating tensors with tf.Varable()**

In [77]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])

changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7])>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7])>)

In [78]:
# Let's try change one of the elements in changeable tensor
# changeable_tensor[0] = 7 # 'ResourceVariable' object does not support item assignment

In [79]:
# How about we try .assign
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7])>

In [80]:
# Let's try change the unchangable tensor
# unchangeable_tensor[0].assign(7) # 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

🔑**Note:** Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### **Creating random tensors**

Random tensors are tensors of some abitrary size which contain random numbers.

In [81]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_2 = random_2.normal(shape=(3,2))

# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### **Shuffle the order of elements in a tensors**

In [82]:
# Shuffle a tensor (valuable when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10, 7],
                          [3, 4],
                          [2, 5]])
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]])>

In [83]:
# Shuffle non-shuffled tensor
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]])>

🔑**Note:** If we want our shuffled tensors to be reproducable, we've got to use the global level random seed as well as the operation level random seed.

### **Other ways to make tensors**

In [84]:
# Create a tenson of all ones
tf.ones([3,5])

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]], dtype=float32)>

In [85]:
# Create a tenson of all ones
tf.zeros([3,4])

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### **Turn NumPy arrays into tensors**

The main difference between NumPy arrays and TensorFlow tensors it that tensors can be run on a GPU (much faster for numerical computing).

In [86]:
# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non capital for vector

import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32)
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [87]:
# You can also turn NumPy arrays into tensors
A = tf.constant(numpy_A)
B = tf.constant(numpy_A, shape=(3,8))
A, B

(<tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24])>,
 <tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24]])>)

### **Getting information from tensors**

When dealing with tensors you probably want to be aware of the following attributes:
| **Attribute** | **Meaning** | **Code** |
| :------------: | :-------:  | :-------:  |
|Shape | The lenght (number of elements) of each of the dimensions of a tensor. | tensor.shape |
|Rank | The number of thensor dimesions. A scalar has rank 0, a vector has rank 1, a tensor has rank n. | tensor.ndim |
|Axis or dimension | A particular dimension of a tensor. | tensor[0], tensor[:, 1], ... |
|Size | A total number of items in the tensor. | tf.size(tensor) |

In [90]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros([2, 2, 3, 4])
rank_4_tensor

<tf.Tensor: shape=(2, 2, 3, 4), dtype=float32, numpy=
array([[[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]],


       [[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]]], dtype=float32)>

In [99]:
# Get various attributes of a tensor
print('Datatype of every element:', rank_4_tensor.dtype)
print('Number of dimensions (rank):', rank_4_tensor.ndim)
print('Shape of tenson:', rank_4_tensor.shape)
print('Elements along the 0 axis:', rank_4_tensor.shape[0])
print('Elements along the last axis:', rank_4_tensor.shape[-1])
print('Total number of elements in a tensor:', tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tenson: (2, 2, 3, 4)
Elements along the 0 axis: 2
Elements along the last axis: 4
Total number of elements in a tensor: 48
